# NICAR Workshop: Machine Learning and NLP

By Jeff Kao, ProPublica

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [ ]:
# !unzip ./data/ira_tweets_csv_hashed.zip -d ./data/

In [ ]:
# !unzip ./data/ira_users_csv_hashed.zip -d ./data/

In [2]:
%%time
df_all = pd.read_csv('./data/ira_tweets_csv_hashed.csv')

<string>:2: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 1min 23s, sys: 5.73 s, total: 1min 29s
Wall time: 1min 27s


In [3]:
df_all.columns

Index(['tweetid', 'userid', 'user_display_name', 'user_screen_name',
       'user_reported_location', 'user_profile_description',
       'user_profile_url', 'follower_count', 'following_count',
       'account_creation_date', 'account_language', 'tweet_language',
       'tweet_text', 'tweet_time', 'tweet_client_name', 'in_reply_to_tweetid',
       'in_reply_to_userid', 'quoted_tweet_tweetid', 'is_retweet',
       'retweet_userid', 'retweet_tweetid', 'latitude', 'longitude',
       'quote_count', 'reply_count', 'like_count', 'retweet_count', 'hashtags',
       'urls', 'user_mentions', 'poll_choices'],
      dtype='object')

In [4]:
df_all['tweet_language'].value_counts()

ru     4853185
en     2997181
und     229392
de       98460
uk       82237
bg       54690
ar       37241
it       20136
fr       12241
es       11922
in        9523
sr        8045
tl        6618
ht        6170
et        5847
tr        3832
sk        3343
nl        3151
da        2987
ro        2980
pt        2698
ja        2653
cy        2591
sl        2307
pl        1804
sv        1759
fi        1611
no        1500
lt        1340
cs        1037
bs         615
lv         505
hu         499
id         466
hi         307
hr         297
ko         288
vi         283
is         268
eu         223
iw          56
zh          56
th          40
fa          37
el          35
hy          23
ur          17
ne           7
si           5
ta           4
km           3
bn           3
mr           2
ml           2
he           2
ps           1
ug           1
iu           1
Name: tweet_language, dtype: int64

In [5]:
df = df_all[df_all['tweet_language'] == 'en']

In [9]:
len(df['userid'].unique())

3077

In [ ]:
df.sample(10)[['userid','tweet_text']]

## Machine Learning: NLP and Unsupervised learning

Supervised learning requires labelled data, but often when we are doing investigations, we don't yet know what we are looking for. Unsupervised learning helps our analysis because we let the numbers take us in the right direction.
* K-means
* HDBSCAN
* IsolationForest

## Twitter Dataset

## Data Exploration

## Natural Language Processing: Turning words to numbers

* bag-of-words (discuss bag-of-characters and n-grams)
* tfidf
* LSI
* word2vec and other deep-learning based language models

## Natural Language Processing: Clustering and outlier detection

## Sample Analysis